In [1]:
import pandas as pd

**Задание 1**

1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
ratings_ammount = ratings.groupby('userId').count()[['rating']].reset_index()
ratings_ammount.rename(columns={'rating': 'ratings_ammount'}, inplace=True)
ratings_ammount.head()

,userId,ratings_ammount
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100


In [5]:
joined_df = ratings.merge(ratings_ammount, how='left', on='userId')
joined_df.head()

,userId,movieId,rating,timestamp,ratings_ammount
0,1,31,2.5,1260759144,20
1,1,1029,3.0,1260759179,20
2,1,1061,3.0,1260759182,20
3,1,1129,2.0,1260759185,20
4,1,1172,4.0,1260759205,20


In [6]:
more_100_ratings = joined_df[joined_df.ratings_ammount >= 100]
more_100_ratings.head()

,userId,movieId,rating,timestamp,ratings_ammount
147,4,10,4.0,949810645,204
148,4,34,5.0,949919556,204
149,4,112,5.0,949810582,204
150,4,141,5.0,949919681,204
151,4,153,4.0,949811346,204


In [7]:
lifetime = more_100_ratings.groupby('userId').agg(['min', 'max'])['timestamp'].reset_index()
lifetime['lifetime'] = lifetime['max'] - lifetime['min']
lifetime.head()

,userId,min,max,lifetime
0,4,949778714,949982274,203560
1,5,1163373044,1163375145,2101
2,8,1154389340,1154474527,85187
3,15,997937239,1469330735,471393496
4,17,1127468587,1127476640,8053


In [8]:
import datetime
str(datetime.timedelta(seconds = lifetime['lifetime'].mean()))

'455 days, 5:29:08.566540'

**Задание  2**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки  
auto - автомобильные перевозки  
air - воздушные перевозки  
client_base - адреса клиентов  

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.


In [9]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [10]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [11]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [12]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [13]:
client_revenue = (client_base
 .merge(rzd, how = 'left')
 .merge(auto, how = 'left')
 .merge(air, how = 'left')
)[['client_id', 'rzd_revenue', 'auto_revenue', 'air_revenue']].fillna(0)
client_revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [14]:
client_revenue_adress = (client_base
 .merge(rzd, how = 'left')
 .merge(auto, how = 'left')
 .merge(air, how = 'left')
).fillna(0)
client_revenue_adress

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


**Ответ**  
Набор координат пользователя может быть получен, например, путем определения геопозиции пользователя или путем определения ip-адреса. Таким образом, для каждого действия пользователя будет предусмотрено 2 координаты - в таблице визитов к каждому визиту (каждая строка - отдельный визит) и в таблице покупок - для каждой покупки (каждая покупка - отдельный визит). Проблема может быть связана с тем, что пользователь откажется давать доступ приложению к своей геопозиции или будет исполь зовать vpn. В таком случае значение поля останется пустым.
Из набора координат можно получить физическое нахождение пользователя (в SQL же есть даже отдельный тип данных с широтой, долготой), которое использовать для построения рекомендательных систем, настройки контекстной рекламы, может быть продвижения оффлайн-бизнеса.
В случае объединения таблиц визитов и покупок и группировки географического положения для каждого пользователя, пользователи, у которых территориальный разброс покупок и визитов будет большой, могут быть, например, исключены из выборки, на основании которой будет тренироваться рекоммендательная система.